# LSSTCam pointing errors
# Looking at the results of homing the mount.

Craig Lage - 18-Apr-25

In [ ]:
import os
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData

In [ ]:
client = makeEfdClient()

## Get the MTMount state changes

In [ ]:
startDay = 20250527
endDay = 20250608
dayObs = startDay
enableDict = {}
homeDict = {}
while dayObs <= endDay:
    start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
    end = Time(f"{dayObsIntToString(calcNextDay(dayObs))}T12:00:00")
    states = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_summaryState",
        columns=['summaryState'],
        begin=start,
        end=end
    )
    enables = states[states['summaryState'] == 2]
    
    homes = getEfdData(
    client,
    "lsst.sal.MTMount.command_homeBothAxes",
    begin=start,
    end=end
)
    print(f"There were {len(enables)} enable events and {len(homes)} home events on {dayObs}")
    enableDict[dayObs] = enables
    homeDict[dayObs] = homes
    dayObs = calcNextDay(dayObs)

In [ ]:
enables = enableDict[20250603]
enables

In [ ]:
homes = homeDict[20250603]
homes

In [ ]:
index = 4
start = Time(enables.index[index]) - TimeDelta(30, format='sec')
end = Time(enables.index[index]) + TimeDelta(360, format='sec')

start = Time("2025-06-25T01:58:00", scale='utc')
end = Time("2025-06-25T01:58:10", scale='utc')
states = getEfdData(
    client,
    "lsst.sal.MTMount.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)

az = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    columns=['actualPosition', 'actualPositionTimestamp'],
    begin=start,
    end=end
)
el = getEfdData(
    client,
    "lsst.sal.MTMount.elevation",
    columns=['actualPosition', 'actualPositionTimestamp'],
    begin=start,
    end=end
)
encoder = getEfdData(
    client,
    "lsst.sal.MTMount.encoder",
    begin=start,
    end=end
)
home = getEfdData(
    client,
    "lsst.sal.MTMount.command_homeBothAxes",
    begin=start,
    end=end
)

In [ ]:
len(home)

In [ ]:
len(states)

In [ ]:
for i in range(4):
    encoder[f'azimuthEncoderAbsolutePosition{i}'].plot()
plt.axvline(Time(enables.index[index]).isot,ls='--', color='red', label='Enabled')
plt.axvline(Time(home.index[0]).isot,ls='--', color='blue', label='Homed')
plt.legend()

In [ ]:
az['actualPosition'].plot(marker='x')
#plt.axvline(Time(enables.index[index]).isot,ls='--', color='red', label='Enabled')
#plt.axvline(Time(home.index[0]).isot,ls='--', color='blue', label='Homed')
plt.legend()

In [ ]:
azValues = np.asarray(az["actualPosition"])
azValTimes = np.asarray(az["actualPositionTimestamp"])
encValues = np.asarray(encoder['azimuthEncoderAbsolutePosition0'])
encTimes = np.asarray(encoder['azimuthEncoderAbsolutePositionTimestamp0'])
encInterp = np.interp(azValTimes, encTimes, encValues)
az['encInterp'] = encInterp
#plt.plot(encInterp, azValues)
#plt.axvline(Time(enables.index[index]).isot,ls='--', color='red', label='Enabled')
#plt.axvline(Time(home.index[0]).isot,ls='--', color='blue', label='Homed')
#plt.legend()

In [ ]:
az['actualPosition'].plot()
az['encInterp'].plot()
#plt.axvline(Time(enables.index[index]).isot,ls='--', color='red', label='Enabled')
#plt.axvline(Time(home.index[0]).isot,ls='--', color='blue', label='Homed')
plt.legend()

In [ ]:
az.plot(x='actualPosition', y='encInterp')

In [ ]:
az.plot(x='actualPosition', y='encInterp')
#plt.xlim(-30, -28)
#plt.ylim(0,1)

In [ ]:
index = 4
start = Time(enables.index[index]) - TimeDelta(30, format='sec')
end = Time(enables.index[index]) + TimeDelta(360, format='sec')

start = Time("2025-06-25T01:56:35", scale='utc')
end = Time("2025-06-25T01:56:50", scale='utc')
states = getEfdData(
    client,
    "lsst.sal.MTMount.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)

az2 = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    columns=['actualPosition', 'actualPositionTimestamp'],
    begin=start,
    end=end
)
el2 = getEfdData(
    client,
    "lsst.sal.MTMount.elevation",
    columns=['actualPosition', 'actualPositionTimestamp'],
    begin=start,
    end=end
)
encoder2 = getEfdData(
    client,
    "lsst.sal.MTMount.encoder",
    begin=start,
    end=end
)
home = getEfdData(
    client,
    "lsst.sal.MTMount.command_homeBothAxes",
    begin=start,
    end=end
)

In [ ]:
azValues2 = np.asarray(az2["actualPosition"])
azValTimes2 = np.asarray(az2["actualPositionTimestamp"])
encValues2 = np.asarray(encoder2['azimuthEncoderAbsolutePosition0'])
encTimes2 = np.asarray(encoder2['azimuthEncoderAbsolutePositionTimestamp0'])
encInterp2 = np.interp(azValTimes2, encTimes2, encValues2)
az2['encInterp'] = encInterp2
#plt.plot(encInterp, azValues)
#plt.axvline(Time(enables.index[index]).isot,ls='--', color='red', label='Enabled')
#plt.axvline(Time(home.index[0]).isot,ls='--', color='blue', label='Homed')
#plt.legend()

In [ ]:
plt.plot(azValues, encInterp, color='blue')
plt.plot(azValues2, encInterp2, color='red')

In [ ]:

start = Time("2025-06-25T03:07:00", scale='utc')
end = Time("2025-06-25T03:11:00", scale='utc')
states = getEfdData(
    client,
    "lsst.sal.MTMount.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)

az3 = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    columns=['actualPosition', 'actualPositionTimestamp'],
    begin=start,
    end=end
)
el2 = getEfdData(
    client,
    "lsst.sal.MTMount.elevation",
    columns=['actualPosition', 'actualPositionTimestamp'],
    begin=start,
    end=end
)
encoder3 = getEfdData(
    client,
    "lsst.sal.MTMount.encoder",
    begin=start,
    end=end
)
home = getEfdData(
    client,
    "lsst.sal.MTMount.command_homeBothAxes",
    begin=start,
    end=end
)

In [ ]:
azValues3 = np.asarray(az3["actualPosition"])
azValTimes3 = np.asarray(az3["actualPositionTimestamp"])
encValues3 = np.asarray(encoder3['azimuthEncoderAbsolutePosition0'])
encTimes3 = np.asarray(encoder3['azimuthEncoderAbsolutePositionTimestamp0'])
encInterp3 = np.interp(azValTimes3, encTimes3, encValues3)
az3['encInterp'] = encInterp3
#plt.plot(encInterp, azValues)
#plt.axvline(Time(enables.index[index]).isot,ls='--', color='red', label='Enabled')
#plt.axvline(Time(home.index[0]).isot,ls='--', color='blue', label='Homed')
#plt.legend()

In [ ]:
plt.plot(azValues, encInterp, color='blue')
plt.plot(azValues2, encInterp2, color='red')
plt.plot(azValues3, encInterp3, color='green')

In [ ]:

start = Time("2025-06-24T03:29:00", scale='utc')
end = Time("2025-06-24T03:32:00", scale='utc')
states = getEfdData(
    client,
    "lsst.sal.MTMount.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)

az4 = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    columns=['actualPosition', 'actualPositionTimestamp'],
    begin=start,
    end=end
)
el4 = getEfdData(
    client,
    "lsst.sal.MTMount.elevation",
    columns=['actualPosition', 'actualPositionTimestamp'],
    begin=start,
    end=end
)
encoder4 = getEfdData(
    client,
    "lsst.sal.MTMount.encoder",
    begin=start,
    end=end
)
home = getEfdData(
    client,
    "lsst.sal.MTMount.command_homeBothAxes",
    begin=start,
    end=end
)

In [ ]:
azValues4 = np.asarray(az4["actualPosition"])
azValTimes4 = np.asarray(az4["actualPositionTimestamp"])
encValues4 = np.asarray(encoder4['azimuthEncoderAbsolutePosition0'])
encTimes4 = np.asarray(encoder4['azimuthEncoderAbsolutePositionTimestamp0'])
encInterp4 = np.interp(azValTimes4, encTimes4, encValues4)
az4['encInterp'] = encInterp4
encInterp34 = np.interp(azValTimes4, encTimes3, encValues3)
az4['encInterp34'] = encInterp34
#plt.plot(encInterp, azValues)
#plt.axvline(Time(enables.index[index]).isot,ls='--', color='red', label='Enabled')
#plt.axvline(Time(home.index[0]).isot,ls='--', color='blue', label='Homed')
#plt.legend()

In [ ]:
len(azValues3)

In [ ]:
#plt.plot(azValues3[1500], encInterp3[1800], color='green')
plt.plot(azValues3[450:470], encInterp3[450:470], color='green', marker='x', label = '2025-06-25T03:07:00')
plt.plot(azValues4[973:993], encInterp4[973:993], color='cyan', ls='--', marker='o', label='2025-06-24T03:29:00')
plt.xlabel("Azimuth (degrees)")
plt.ylabel("azimuthEncoderAbsolutePosition0")
plt.legend()
plt.savefig("/home/c/cslage/u/MTMount/mount_plots/Encoder_vs_Azimuth_25Jun25.png")

In [ ]:
plt.plot(azValues3[0:500], encInterp3[0:500], color='green')

In [ ]:
plt.plot(azValues, encInterp, color='blue')
plt.plot(azValues2, encInterp2, color='red')
plt.plot(azValues3, encInterp3, color='green')
plt.plot(azValues4, encInterp4, color='cyan', ls='--')
plt.xlim(80, 110)
plt.ylim(80,110)

In [ ]:
len(azValues3)

In [ ]:
from lsst.summit.utils.tmaUtils import TMAEventMaker
eventMaker = TMAEventMaker()

In [ ]:
dayObs = 20250531
dayObs = 20250601
events = eventMaker.getEvents(dayObs)

In [ ]:
myEvents = [e for e in events if ((e.type.name=='SLEWING') and (e.duration > 100.0) and (e.duration < 150.0))]
len(myEvents)

In [ ]:
myEvents[0]

In [ ]:
event = myEvents[0]
az = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    columns=['actualPosition', 'actualPositionTimestamp'],
    event=event, 
    prePadding = -2.0,
    postPadding = -2.0
)
el = getEfdData(
    client,
    "lsst.sal.MTMount.elevation",
    columns=['actualPosition', 'actualPositionTimestamp'],
    event=event, 
    prePadding = -2.0,
    postPadding = -2.0
)
encoder = getEfdData(
    client,
    "lsst.sal.MTMount.encoder",
    event=event, 
    prePadding = -2.0,
    postPadding = -2.0
)

azimuthPosition = encoder['azimuthEncoderPosition0']
azimuthPosition += encoder['azimuthEncoderPosition1']
azimuthPosition += encoder['azimuthEncoderPosition2']
azimuthPosition += encoder['azimuthEncoderPosition3']
azimuthPosition /= 4.0
encoder['azimuthPosition'] = azimuthPosition

elevationPosition = encoder['elevationEncoderPosition0']
elevationPosition += encoder['elevationEncoderPosition1']
elevationPosition += encoder['elevationEncoderPosition2']
elevationPosition += encoder['elevationEncoderPosition3']
elevationPosition /= 4.0
encoder['elevationPosition'] = elevationPosition

azValues = np.asarray(az["actualPosition"])
azValTimes = np.asarray(az["actualPositionTimestamp"])
encValues = np.asarray(encoder['azimuthPosition'])
encTimes = np.asarray(encoder['azimuthEncoderPositionTimestamp0'])
azEncInterp = np.interp(azValTimes, encTimes, encValues)
az['encoderPosition'] = azEncInterp

elValues = np.asarray(el["actualPosition"])
elValTimes = np.asarray(el["actualPositionTimestamp"])
encValues = np.asarray(encoder['elevationPosition'])
encTimes = np.asarray(encoder['elevationEncoderPositionTimestamp0'])
elEncInterp = np.interp(elValTimes, encTimes, encValues)
el['encoderPosition'] = elEncInterp


In [ ]:
fig, ax = plt.subplots(1,1)

ax.scatter(azValues, azEncInterp)
fit = np.polyfit(azValues, azEncInterp, 1)
print(fit)
ax.text(0.1, 0.8, f"Slope = {fit[0]:.6f}", transform=ax.transAxes)
ax.text(0.1, 0.7, f"Offset = {(fit[1]*3600):.2f} arcseconds", transform=ax.transAxes)

In [ ]:
fig, ax = plt.subplots(1,1)

ax.scatter(azValues, azEncInterp)
fit = np.polyfit(azValues, azEncInterp, 1)
print(fit)
ax.text(0.1, 0.8, f"Slope = {fit[0]:.6f}", transform=ax.transAxes)
ax.text(0.1, 0.7, f"Offset = {(fit[1]*3600):.2f} arcseconds", transform=ax.transAxes)